# Economics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os
from fredapi import Fred
from functions import humanize_number

In [ ]:
# Matplotlib Style Configuration
# Change this line to use different graph styles

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('default')            # Clean white background

# Custom style settings (optional)
plt.rcParams.update({
    'figure.figsize': (12, 8),      # Default figure size
    'font.size': 10,                # Default font size
    'axes.labelsize': 12,           # Axis label size
    'axes.titlesize': 14,           # Title size
    'xtick.labelsize': 10,          # X-tick label size
    'ytick.labelsize': 10,          # Y-tick label size
    'legend.fontsize': 10,          # Legend font size
    'figure.dpi': 200,              # Resolution
})

# List available styles
#for i, style in enumerate(sorted(plt.style.available), 1):
    #print(f"  {i:2d}. {style}")

In [ ]:
# Set date range for analysis (change these variables to adjust the date range)
start_date = '2000-01-01'  # Start date for filtering data (YYYY-MM-DD format)

In [ ]:
# Function to add grey recession shading to plots
def add_recession_shading(ax):
    try:
        recession = fred.get_series('USREC')
        recession = recession.loc[start_date:]
        # Find recession periods (where USREC == 1)
        in_recession = False
        start_date_rec = None
        for date, value in recession.items():
            if value == 1 and not in_recession:
                start_date_rec = date
                in_recession = True
            elif value == 0 and in_recession:
                ax.axvspan(start_date_rec, date, color='grey', alpha=0.3)
                in_recession = False
        # Handle if still in recession at end
        if in_recession:
            ax.axvspan(start_date_rec, recession.index[-1], color='grey', alpha=0.3)
    except Exception as e:
        print(f"Could not add recession shading: {e}")

In [ ]:
# Fetch US employment data (PAYEMS: All Employees, Total Nonfarm)
employment = fred.get_series('PAYEMS')

# Fetch US hires data (JTSHIR: Job Openings and Labor Turnover Survey)
hires = fred.get_series('JTSHIR')

# Filter to specified date range
employment = employment.loc[start_date:]
hires = hires.loc[start_date:]

# Plot the data
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# Plot employment
axes[0].plot(employment.index, employment.values)
axes[0].set_title('US Employment (All Employees, Total Nonfarm)')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Number of Employees (Thousands)')
axes[0].grid(True)
add_recession_shading(axes[0])
axes[0].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: x))

# Plot hires
axes[1].plot(hires.index, hires.values)
axes[1].set_title('US Hires (Job Openings and Labor Turnover Survey)')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Thousands of Persons')
axes[1].grid(True)
add_recession_shading(axes[1])
axes[1].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: x))

plt.tight_layout()
plt.show()

In [ ]:
# Fetch US trade data (Goods and Services)
exports = fred.get_series('EXPGSC1')  # Exports
imports = fred.get_series('IMPGSC1')  # Imports
net_imports = imports - exports  # Net Imports (Imports - Exports)

# Filter to specified date range
exports = exports.loc[start_date:]
imports = imports.loc[start_date:]
net_imports = net_imports.loc[start_date:]

# Create a grid layout: Net imports on top, spanning both columns; Exports and Imports below
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(15, 10))
gs = gridspec.GridSpec(2, 2, figure=fig)

# Net imports on top, spanning both columns
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(net_imports.index, net_imports.values)
ax1.set_title('US Net Imports (Goods and Services)')
ax1.set_xlabel('Date')
ax1.set_ylabel('Billions of Dollars')
ax1.grid(True)
add_recession_shading(ax1)

# Exports bottom left
ax2 = fig.add_subplot(gs[1, 0])
ax2.plot(exports.index, exports.values)
ax2.set_title('US Exports (Goods and Services)')
ax2.set_xlabel('Date')
ax2.set_ylabel('Billions of Dollars')
ax2.grid(True)
add_recession_shading(ax2)

# Imports bottom right
ax3 = fig.add_subplot(gs[1, 1])
ax3.plot(imports.index, imports.values)
ax3.set_title('US Imports (Goods and Services)')
ax3.set_xlabel('Date')
ax3.set_ylabel('Billions of Dollars')
ax3.grid(True)
add_recession_shading(ax3)

plt.tight_layout()
plt.show()

In [ ]:
# Plot the specific series DRCCLACBS and DRCLACBN side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# Plot DRCCLACBS
try:
    series_data1 = fred.get_series('DRCCLACBS')
    series_data1 = series_data1.loc[start_date:]
    print("Series DRCCLACBS fetched successfully")
    
    # Get series info
    series_info1 = fred.get_series_info('DRCCLACBS')
    title1 = series_info1.get('title', 'DRCCLACBS')
    units1 = series_info1.get('units', 'Value')
    
    axes[0].plot(series_data1.index, series_data1.values)
    axes[0].set_title(f'{title1}')
    axes[0].set_xlabel('Date')
    axes[0].set_ylabel(units1)
    axes[0].grid(True)
    add_recession_shading(axes[0])
    
except Exception as e:
    print(f"Series DRCCLACBS not available: {e}")
    axes[0].text(0.5, 0.5, 'DRCCLACBS\nNot Available', transform=axes[0].transAxes, ha='center', va='center')
    axes[0].set_title('DRCCLACBS')

# Plot DRCLACBN
try:
    series_data2 = fred.get_series('DRCLACBN')
    series_data2 = series_data2.loc[start_date:]
    print("Series DRCLACBN fetched successfully")
    
    # Get series info
    series_info2 = fred.get_series_info('DRCLACBN')
    title2 = series_info2.get('title', 'DRCLACBN')
    units2 = series_info2.get('units', 'Value')
    
    axes[1].plot(series_data2.index, series_data2.values)
    axes[1].set_title(f'{title2}')
    axes[1].set_xlabel('Date')
    axes[1].set_ylabel(units2)
    axes[1].grid(True)
    add_recession_shading(axes[1])
    
except Exception as e:
    print(f"Series DRCLACBN not available: {e}")
    axes[1].text(0.5, 0.5, 'DRCLACBN\nNot Available', transform=axes[1].transAxes, ha='center', va='center')
    axes[1].set_title('DRCLACBN')

plt.tight_layout()
plt.show()

## Other

In [ ]:
# Plot the specific series APU000072610
try:
    series_data = fred.get_series('APU000072610')
    # Use a more recent start date for this series to avoid parsing issues
    series_data = series_data.loc[start_date:]
    print("Series APU000072610 fetched successfully")
    
    # Get series info
    series_info = fred.get_series_info('APU000072610')
    title = series_info.get('title', 'APU000072610')
    units = series_info.get('units', 'Value')
    
    # Plot
    plt.figure(figsize=(12, 6))
    series_data.plot()
    plt.title(f'{title}')
    plt.xlabel('Date')
    plt.ylabel(units)
    plt.grid(True)
    add_recession_shading(plt.gca())
    plt.show()
    
except Exception as e:
    print(f"Series APU000072610 not available: {e}")

In [ ]:
# Plot inflation data (Consumer Price Index)
try:
    inflation = fred.get_series('CPIAUCSL')
    inflation = inflation.loc[start_date:]
    print("Series CPIAUCSL fetched successfully")
    
    # Calculate year-over-year inflation rate
    inflation_rate = inflation.pct_change(12) * 100
    
    # Plot
    plt.figure(figsize=(12, 6))
    inflation_rate.plot()
    plt.title('US Inflation Rate (Year-over-Year % Change)')
    plt.xlabel('Date')
    plt.ylabel('Percent')
    plt.grid(True)
    add_recession_shading(plt.gca())
    plt.show()
    
except Exception as e:
    print(f"Series CPIAUCSL not available: {e}")

In [ ]:
# Plot sector-specific inflation rates in a 3x3 grid
import concurrent.futures

sector_series = [
    'CPIUFDSL',  # Food
    'CPIENGSL',  # Energy
    'CUSR0000SAH1',  # Housing
    'CUSR0000SAT1',  # Transportation
    'CUSR0000SAM1',  # Medical care
    'CUSR0000SAA1',  # Apparel
    'CUSR0000SEEB',  # Education and communication
    'CUSR0000SERA',  # Recreation
    'CUSR0000SEHF'  # Other goods and services
]

sector_names = [
    'Food',
    'Energy',
    'Housing',
    'Transportation',
    'Medical Care',
    'Apparel',
    'Education and Communication',
    'Recreation',
    'Other Goods and Services'
]

# Fetch all series concurrently
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(fred.get_series, series): (series, name) for series, name in zip(sector_series, sector_names)}
    results = {}
    for future in concurrent.futures.as_completed(futures):
        series, name = futures[future]
        try:
            data = future.result()
            results[name] = data
        except Exception as e:
            print(f"Series {series} not available: {e}")
            results[name] = None

fig, axes = plt.subplots(3, 3, figsize=(15, 15))

for i, name in enumerate(sector_names):
    data = results[name]
    if data is not None:
        data = data.loc[start_date:]
        rate = data.pct_change(12) * 100
        axes.flat[i].plot(rate.index, rate.values)
        axes.flat[i].set_title(name)
        axes.flat[i].set_xlabel('Date')
        axes.flat[i].set_ylabel('YoY % Change')
        axes.flat[i].grid(True)
        add_recession_shading(axes.flat[i])
    else:
        axes.flat[i].text(0.5, 0.5, f'{name}\nNot Available', transform=axes.flat[i].transAxes, ha='center', va='center')
        axes.flat[i].set_title(name)

plt.tight_layout()
plt.show()